In [2]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import time
import random

In [3]:
def scrapePainting(url):
    r = requests.get(url)
    print r, url
    b = BeautifulSoup(r.text, 'html.parser')
    
    url_id = r.url.split('/')[-1]
    
    artist = b.find('a', {'id':'fine-ART-ProductLabelArtistNameLink'})
    if artist is not None:
        artist = b.find('a', {'id':'fine-ART-ProductLabelArtistNameLink'}).text
        
    title = b.find('span', {'id': 'fineArtTitle'})
    if title is not None:
        title = b.find('span', {'id': 'fineArtTitle'}).text
        
    image = re.findall('http://ecx.images-amazon.com/images/I/.*\.jpg', r.text)
    if image is not None:
        image = re.findall('http://ecx.images-amazon.com/images/I/.*\.jpg', r.text)
        
    price = b.find('span', {'id': 'priceblock_ourprice'})
    if price is not None:
        price = b.find('span', {'id': 'priceblock_ourprice'}).text.replace('$','').replace(',', '')
        if '.' in price:
            price = float(price.replace('.', ''))/100
        else:
            price = float(price)
            
    desc = b.find('div', {'id': 'productDescription_feature_div'})
    if desc is not None:
        desc = b.find('div', {'id': 'productDescription_feature_div'}).getText().replace('\n','')
        
    height = None
    width = None
    size = b.find('span', {'id': 'mnba_buybox_size'})
    if size is not None:
        size = [a.strip() for a in b.find('span', {'id': 'mnba_buybox_size'}).text.split('x')]
        if len(size) == 2:
            height = float(size[0].replace('in.','').strip())
            width = float(size[1].replace('in.','').strip())
            depth = 1.
        elif len(size) == 3:
            height = float(size[0].replace('in.','').strip())
            width = float(size[1].replace('in.','').strip())
            depth = float(size[2].replace('in.','').strip())

    return {'url_id': url_id, 'artist': artist, 'title':title, 'image': image, 'price': price, 
            'description':desc, 'height': height, 'width': width, 'size':size}

In [4]:
scrapePainting('http://www.amazon.com/Fire-Hydrant/dp/B01EKBYJYW/')

<Response [503]> http://www.amazon.com/Fire-Hydrant/dp/B01EKBYJYW/


{'artist': None,
 'description': None,
 'height': None,
 'image': [],
 'price': None,
 'size': None,
 'title': None,
 'url_id': u'B01EKBYJYW',
 'width': None}

In [5]:
for page in range(1,4):
    
    page_url = "http://www.amazon.com/s/ref=sr_pg_3?rh=n%3A4991425011%2Cn%3A%214991426011%2Cn%3A6685269011%2Cn%3A6685289011&page="+str(page)+"&ie=UTF8&qid=1462467907"
    r = requests.get(page_url)
    r.text

    b = BeautifulSoup(r.text, 'html.parser')

    url_1 = []
    for link in b.findAll('a', {'class':"a-link-normal a-text-normal"}):
        url_1.append(link.get('href'))
    
    url_list = url_1[::2]

    paintings = []
    for url in url_list:
        try:
            paintings.append(scrapePainting(url))
        except Exception as error:
            print error
        time.sleep(random.randint(5,20))

df = pd.DataFrame(paintings)

<Response [503]> http://www.amazon.com/12x18-in-Jack-Androvich-Surfboards/dp/B00EZH6DM8
<Response [200]> http://www.amazon.com/For-sale/dp/B015EOYYEE
<Response [200]> http://www.amazon.com/Steve-Fireman-Beverly-Shores-IN/dp/B00IRTE78C
<Response [200]> http://www.amazon.com/30x40-Simon-Carruthers-Untitled-Pallets/dp/B00LLQ152E
<Response [200]> http://www.amazon.com/Let-Go/dp/B00SVIN0W8
<Response [200]> http://www.amazon.com/Prince/dp/B016VGH73O
<Response [200]> http://www.amazon.com/Fire-Hydrant/dp/B01EKBYJYW
<Response [503]> http://www.amazon.com/Kathy-on-a-Windy-Boat/dp/B00H9MAHUK
<Response [200]> http://www.amazon.com/Ayrton-Senna-F1/dp/B016VGE62E
<Response [503]> http://www.amazon.com/Toy/dp/B00SVITN6A
<Response [200]> http://www.amazon.com/Nude-in-Water/dp/B016VGKAW4
<Response [200]> http://www.amazon.com/Half-Dome-Breaking-Storm/dp/B00NNA11KC
<Response [503]> http://www.amazon.com/fish-splash/dp/B016VGCLNA
<Response [503]> http://www.amazon.com/28x28-Tom-Chambers-Ashly-Deer/dp/B00

In [8]:
df

,artist,description,height,image,price,size,title,url_id,width
0,None,None,NaN,[],NaN,None,None,B010W24SIU,NaN
1,Scott Takeda,Japan is the birthplace of my great-grandparen...,18.00,[http://ecx.images-amazon.com/images/I/610q%2B...,125.0,"[18, 0.01, 12 in.]","Tree,",B017C8HICK,0.01
2,Stuart Zaro,Waiting is the iconic image for the more than ...,NaN,[http://ecx.images-amazon.com/images/I/61L92O6...,500.0,None,"Waiting,",B00VQWYZY8,NaN
3,None,None,NaN,[],NaN,None,None,B00EI3ZLI6,NaN
4,Linda Leiberman,None,NaN,[http://ecx.images-amazon.com/images/I/51u9IGi...,560.0,None,"Down Boy,",B016VGLKX2,NaN
5,Slim Aarons,C-Print,23.00,[http://ecx.images-amazon.com/images/I/51%2BNX...,4500.0,"[23, 15.2 in.]","Hotel Taormina Pool,",B00E7SEH32,15.20
6,Michael Verlangieri,Maui Beach Sunset by Michael Verlangieri. Limi...,21.50,[http://ecx.images-amazon.com/images/I/51xX03M...,140.0,"[21.5, 28 in.]","Maui Beach Sunset Fine Art Photo Hawaii,",B00S61NUYI,28.00
7,Bruce Mozert,Signed in pencil on print verso. Artist copyri...,8.00,[http://ecx.images-amazon.com/images/I/51uidA1...,1400.0,"[8, 10 in.]",Silver Springs Underwater (Sun Bathing),B00WRLFEJM,10.00
8,John Vlahakis,"Flora, stamen, blue, yellow, green, black, clo...",24.00,[http://ecx.images-amazon.com/images/I/51ITW1l...,1350.0,"[24, 30 in.]",Sperm,B00H277GX8,30.00
9,None,None,NaN,[],NaN,None,None,B00NC7AQN4,NaN


In [ ]:
df.to_csv("paintings_first_df.csv", index=False)